# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files("C:\\Users\\Kareem.Rady\\Desktop\\Courses_udacity\\UDEND\\Project1\\data\\song_data")

In [5]:
song_files

['C:\\Users\\Kareem.Rady\\Desktop\\Courses_udacity\\UDEND\\Project1\\data\\song_data\\A\\A\\A\\TRAAAAW128F429D538.json',
 'C:\\Users\\Kareem.Rady\\Desktop\\Courses_udacity\\UDEND\\Project1\\data\\song_data\\A\\A\\A\\TRAAABD128F429CF47.json',
 'C:\\Users\\Kareem.Rady\\Desktop\\Courses_udacity\\UDEND\\Project1\\data\\song_data\\A\\A\\A\\TRAAADZ128F9348C2E.json',
 'C:\\Users\\Kareem.Rady\\Desktop\\Courses_udacity\\UDEND\\Project1\\data\\song_data\\A\\A\\A\\TRAAAEF128F4273421.json',
 'C:\\Users\\Kareem.Rady\\Desktop\\Courses_udacity\\UDEND\\Project1\\data\\song_data\\A\\A\\A\\TRAAAFD128F92F423A.json',
 'C:\\Users\\Kareem.Rady\\Desktop\\Courses_udacity\\UDEND\\Project1\\data\\song_data\\A\\A\\A\\TRAAAMO128F1481E7F.json',
 'C:\\Users\\Kareem.Rady\\Desktop\\Courses_udacity\\UDEND\\Project1\\data\\song_data\\A\\A\\A\\TRAAAMQ128F1460CD3.json',
 'C:\\Users\\Kareem.Rady\\Desktop\\Courses_udacity\\UDEND\\Project1\\data\\song_data\\A\\A\\A\\TRAAAPK128E0786D96.json',
 'C:\\Users\\Kareem.Rady\\Deskto

In [6]:
filepath = song_files[0]

In [7]:
df = pd.read_json(filepath, lines=True)
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARD7TVE1187B99BFB1,NaN,California - LA,NaN,Casual,218.93179,1,SOMZWCG12A8C13C480,I Didn't Mean To,0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [8]:
song_data = df[["song_id", "title", "artist_id", "year", "duration"]].values
song_data = list(song_data[0])
song_data

['SOMZWCG12A8C13C480', "I Didn't Mean To", 'ARD7TVE1187B99BFB1', 0, 218.93179]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [10]:
song_table_insert

' INSERT INTO songs (song_id, title, artist_id ,                                             year, duration)                          VALUES (%s, %s, %s, %s, %s)\n'

In [11]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [12]:
artist_data = df[["artist_id", "artist_name", "artist_location", "artist_latitude", "artist_longitude"]].fillna(0).values
artist_data = list(artist_data[0])

In [13]:
artist_data

['ARD7TVE1187B99BFB1', 'Casual', 'California - LA', 0.0, 0.0]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [14]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [15]:
log_files = get_files(r"C:\Users\Kareem.Rady\Desktop\Courses_udacity\UDEND\Project1\data\log_data")

In [16]:
filepath = log_files[0]

In [17]:
df = pd.read_json(filepath, lines=True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Walter,M,0,Frye,NaN,free,"San Francisco-Oakland-Hayward, CA",GET,Home,1540919166796,38,None,200,1541105830796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",39
1,None,Logged In,Kaylee,F,0,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Home,1540344794796,139,None,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
3,None,Logged In,Kaylee,F,2,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Upgrade,1540344794796,139,None,200,1541106132796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Flat 55,200,1541106352796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [18]:
df = df[df.page == "NextSong"]
df.head(10)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Flat 55,200,1541106352796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
5,Tamba Trio,Logged In,Kaylee,F,4,Summers,177.18812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Quem Quiser Encontrar O Amor,200,1541106496796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
6,The Mars Volta,Logged In,Kaylee,F,5,Summers,380.42077,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Eriatarka,200,1541106673796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
7,Infected Mushroom,Logged In,Kaylee,F,6,Summers,440.26730,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Becoming Insane,200,1541107053796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
8,Blue October / Imogen Heap,Logged In,Kaylee,F,7,Summers,241.39710,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Congratulations,200,1541107493796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
9,Girl Talk,Logged In,Kaylee,F,8,Summers,160.15628,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Once again,200,1541107734796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
10,Black Eyed Peas,Logged In,Sylvie,F,0,Cruz,214.93506,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",PUT,NextSong,1540266185796,9,Pump It,200,1541108520796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",10
12,Fall Out Boy,Logged In,Ryan,M,1,Smith,200.72444,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1541016707796,169,Nobody Puts Baby In The Corner,200,1541109125796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26
13,M.I.A.,Logged In,Ryan,M,2,Smith,233.71710,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1541016707796,169,Mango Pickle Down River (With The Wilcannia Mob),200,1541109325796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26


In [19]:
t = pd.to_datetime(df["ts"], unit='ms')
t.head()

2   2018-11-01 21:01:46.796
4   2018-11-01 21:05:52.796
5   2018-11-01 21:08:16.796
6   2018-11-01 21:11:13.796
7   2018-11-01 21:17:33.796
Name: ts, dtype: datetime64[ns]

In [19]:
dir(t.dt)

['__bytes__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__unicode__',
 '__weakref__',
 '_accessors',
 '_add_delegate_accessors',
 '_constructor',
 '_delegate_method',
 '_delegate_property_get',
 '_delegate_property_set',
 '_deprecations',
 '_dir_additions',
 '_dir_deletions',
 '_freeze',
 '_get_values',
 '_reset_cache',
 'ceil',
 'date',
 'day',
 'day_name',
 'dayofweek',
 'dayofyear',
 'days_in_month',
 'daysinmonth',
 'floor',
 'freq',
 'hour',
 'is_leap_year',
 'is_month_end',
 'is_month_start',
 'is_quarter_end',
 'is_quarter_start',
 'is_year_end',
 'is_year_start',
 'microsecond',
 'minute',
 'month',
 'month_name',
 'nanosecond',
 'normalize',
 'quarter',
 'round',
 'sec

In [20]:
t

2    2018-11-01 21:01:46.796
4    2018-11-01 21:05:52.796
5    2018-11-01 21:08:16.796
6    2018-11-01 21:11:13.796
7    2018-11-01 21:17:33.796
8    2018-11-01 21:24:53.796
9    2018-11-01 21:28:54.796
10   2018-11-01 21:42:00.796
12   2018-11-01 21:52:05.796
13   2018-11-01 21:55:25.796
14   2018-11-01 22:23:14.796
Name: ts, dtype: datetime64[ns]

In [21]:
time_data = (df['ts'], t.dt.hour, t.dt.day, t.dt.weekofyear, t.dt.month, t.dt.year, t.dt.weekday)
column_labels = ("time_stamp", "hour", "day", "weekofyear", "month", "year", "weekday")
time_data

(2     1541106106796
 4     1541106352796
 5     1541106496796
 6     1541106673796
 7     1541107053796
 8     1541107493796
 9     1541107734796
 10    1541108520796
 12    1541109125796
 13    1541109325796
 14    1541110994796
 Name: ts, dtype: int64, 2     21
 4     21
 5     21
 6     21
 7     21
 8     21
 9     21
 10    21
 12    21
 13    21
 14    22
 Name: ts, dtype: int64, 2     1
 4     1
 5     1
 6     1
 7     1
 8     1
 9     1
 10    1
 12    1
 13    1
 14    1
 Name: ts, dtype: int64, 2     44
 4     44
 5     44
 6     44
 7     44
 8     44
 9     44
 10    44
 12    44
 13    44
 14    44
 Name: ts, dtype: int64, 2     11
 4     11
 5     11
 6     11
 7     11
 8     11
 9     11
 10    11
 12    11
 13    11
 14    11
 Name: ts, dtype: int64, 2     2018
 4     2018
 5     2018
 6     2018
 7     2018
 8     2018
 9     2018
 10    2018
 12    2018
 13    2018
 14    2018
 Name: ts, dtype: int64, 2     3
 4     3
 5     3
 6     3
 7     3
 8     3
 9     3
 

In [32]:
time_df = pd.DataFrame(time_data, column_labels)
time_df.head(10)

,2,4,5,6,7,8,9,10,12,13,14
time_stamp,1541106106796,1541106352796,1541106496796,1541106673796,1541107053796,1541107493796,1541107734796,1541108520796,1541109125796,1541109325796,1541110994796
hour,21,21,21,21,21,21,21,21,21,21,22
day,1,1,1,1,1,1,1,1,1,1,1
weekofyear,44,44,44,44,44,44,44,44,44,44,44
month,11,11,11,11,11,11,11,11,11,11,11
year,2018,2018,2018,2018,2018,2018,2018,2018,2018,2018,2018
weekday,3,3,3,3,3,3,3,3,3,3,3


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [33]:
for i, row in time_df.iterrows():
  print(list(row))  

[1541106106796, 1541106352796, 1541106496796, 1541106673796, 1541107053796, 1541107493796, 1541107734796, 1541108520796, 1541109125796, 1541109325796, 1541110994796]
[21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 22]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44]
[11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11]
[2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018]
[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]


In [34]:
time_table_insert

' INSERT INTO time (time_stamp, hour, day, weekofyear,                                            month, year, weekday)                         VALUES (%s, %s, %s, %s, %s, %s, %s)\n'

In [35]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

TypeError: not all arguments converted during string formatting

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [ ]:
user_df = 

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [ ]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [ ]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = ()
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [ ]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.